In [26]:
import pandas as pd

In [27]:
# ler dataset
mel_data = pd.read_csv(r"C:\Users\HP\Desktop\2022\kaggle\Melbourne_Housing\melb_data.csv")

In [28]:
# Filtrar linhas sem preço de venda
filter_mel_data = mel_data.dropna(axis=0)

In [29]:
# Escolhe target e colunas
y = filter_mel_data.Price
mel_colunas = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filter_mel_data[mel_colunas]

In [30]:
y.head()

1    1035000.0
2    1465000.0
4    1600000.0
6    1876000.0
7    1636000.0
Name: Price, dtype: float64

In [31]:
X.head()

,Rooms,Bathroom,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
1,2,1.0,156.0,79.0,1900.0,-37.8079,144.9934
2,3,2.0,134.0,150.0,1900.0,-37.8093,144.9944
4,4,1.0,120.0,142.0,2014.0,-37.8072,144.9941
6,3,2.0,245.0,210.0,1910.0,-37.8024,144.9993
7,2,1.0,256.0,107.0,1890.0,-37.8060,144.9954


In [32]:
pip install sklearn

In [33]:
# Importa a biblioteca DecisionTreeeRegressor
from sklearn.tree import DecisionTreeRegressor

# Define modelo. 
melbourne_model = DecisionTreeRegressor()

# Ajusta modelo
melbourne_model.fit(X, y)

DecisionTreeRegressor()

In [34]:
# Calcula a media de erro absoluto
from sklearn.metrics import mean_absolute_error

previsao_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, previsao_home_prices)

434.71594577146544

In [35]:
# Usando validação com train_test_split
from sklearn.model_selection import train_test_split

# split data em training and validation data, para ambas as colunas e target
# o split se baseia na geração de números randômicos. Fornecer valor numérico para random_state 
# garantira que teremos a mesma variação quando executar o este script
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

# Define modelo
melbourne_model.fit(train_X, train_y)

# Encontra previsao de preços com validação dos dados
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))


260625.0335700452


In [39]:
# importa biblioteca para o cálculo do mae
from sklearn.metrics import mean_absolute_error
# Funçã para calculo do mae (média erro absoluto)
def get_mae(max_leaf_nodes, train_X, val_X, train_Y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state = 0)
    model.fit(train_X, train_Y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [40]:
# Compara o MAE com diferentes valores de max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print ("Máximo de nós: %d \t\t Média de Erro Absoluto:   %d" %(max_leaf_nodes, my_mae))

Máximo de nós: 5 		 Média de Erro Absoluto:   347380
Máximo de nós: 50 		 Média de Erro Absoluto:   258171
Máximo de nós: 500 		 Média de Erro Absoluto:   243495
Máximo de nós: 5000 		 Média de Erro Absoluto:   254983


In [41]:
# Compara para achar o melhor valor de erro absoluto de acordo com a quantidade de nós
candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
# Write loop to find the ideal tree size from candidate_max_leaf_nodes
scores = {leaf_size:get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}
print (scores)
# Store the best value of max_leaf_nodes (it will be either 5, 25, 50, 100, 250 or 500)
best_tree_size =  min(scores, key=scores.get)
best_leaf_nodes = scores[best_tree_size]
print ("Máximo de nós: %d \t\t Média de Erro Absoluto:   %d" %(best_leaf_nodes, best_tree_size))

{5: 347380.33833344496, 25: 271044.0317616614, 50: 258171.21202406782, 100: 248734.2321681268, 250: 247206.39987469296, 500: 243495.96361790417}
Máximo de nós: 243495 		 Média de Erro Absoluto:   500


In [42]:
# Preenche o modelo com o melhor valor encontrado
final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=1)
# Ajusta o modelo final
final_model.fit(X, y)

DecisionTreeRegressor(max_leaf_nodes=500, random_state=1)